In [1]:
# Data handling
import numpy as np
import pandas as pd
import re

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import graphviz
import optuna
import optuna.visualization as vis
%matplotlib inline

# Stats
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro, norm

# Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, export_graphviz

# Metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

# typing
from typing import Dict,List

# os
import os

# time
import time

# warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import yaml


try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

### 1. Loading

In [50]:
df_scraped = pd.read_csv(config["data"]["raw"]["file_scraped"])
df_verified = pd.read_csv(config["data"]["raw"]["file_verified"])
df_vulnerable = pd.read_csv(config["data"]["raw"]["file_vulnerable"])
df_scraped.head()

,name,id,publisher,version,description,categories,tags,install_count,rating,last_updated
0,Python,f1f59ae4-9318-4f3c-a9b5-81b2eaa5f8a5,ms-python,2025.11.2025072901,Python language support with extension access ...,Programming Languages;Debuggers;Data Science;M...,__ext_j2;__ext_jinja2;__web_extension;debugger...,177844528,4.192869,2025-07-29T10:50:24.31Z
1,Pylance,364d2426-116a-433a-a5d8-a5098dc3afbd,ms-python,2025.7.100,"A performant, feature-rich language server for...",Programming Languages,__web_extension;json;python,146473811,3.007722,2025-07-30T23:10:58.657Z
2,Jupyter,6c2f1801-1e7f-45b2-9b5c-7782f1e076e8,ms-toolsai,2025.7.2025073101,"Jupyter notebook support, interactive programm...",Extension Packs;Data Science;Machine Learning;...,__ext_jl;__web_extension;debuggers;interactive...,93931981,2.700599,2025-07-31T09:54:06Z
3,C/C++,690b692e-e8a9-493f-b802-8089d50ac1b2,ms-vscode,1.26.3,"C/C++ IntelliSense, debugging, and code browsing.",Programming Languages;Snippets;Linters;Debugge...,C;C++;cpp;cuda-cpp;debuggers;IntelliSense;json...,85413454,3.426871,2025-06-25T19:02:16.243Z
4,Python Debugger,4bd5d2c9-9d65-401a-b0b2-7498d9f17615,ms-python,2025.11.2025072901,Python Debugger extension using debugpy.,Debuggers,debugger;debuggers;debugpy;python,84033338,4.615385,2025-07-29T10:38:32.423Z


In [55]:
raw_files = {"scraped": df_scraped, "verified": df_verified, "vulnerable": df_vulnerable}
for key, val in raw_files.items():
        print (f"Dimension of file '{key}': {val.shape}")


Dimension of file 'scraped': (81946, 10)
Dimension of file 'verified': (50924, 5)
Dimension of file 'vulnerable': (32135, 12)


In [52]:
for key, val in raw_files.items():
        print (f"Null values of '{key}': {val.isna().sum()}")
        print()

Null values of 'scraped': name                 0
id                   0
publisher            1
version              0
description      11993
categories           0
tags             31388
install_count        0
rating               0
last_updated         0
dtype: int64

Null values of 'verified': Extension Name        0
Publisher             0
Verified              0
Install Count         0
Source Code       11025
dtype: int64

Null values of 'vulnerable': Extension Name                      5
Repository Link                     5
Repository Name                     0
Total Vulnerabilities               0
Critical                            0
High                                0
Medium                              0
Low                                 0
Critical Vulnerability Names    33052
High Vulnerability Names        32466
Medium Vulnerability Names      31388
Low Vulnerability Names         31155
dtype: int64



In [22]:
for key, val in raw_files.items():
    print(f"{key}")
    for col in val.columns:
        print (f"Duplicated in column '{col}' of '{key}': {val[col].duplicated().sum()}")
    print()    

scraped
Duplicated in column 'name' of 'scraped': 18068
Duplicated in column 'id' of 'scraped': 16273
Duplicated in column 'publisher' of 'scraped': 39541
Duplicated in column 'version' of 'scraped': 95275
Duplicated in column 'description' of 'scraped': 29244
Duplicated in column 'categories' of 'scraped': 97509
Duplicated in column 'tags' of 'scraped': 66376
Duplicated in column 'install_count' of 'scraped': 83061
Duplicated in column 'rating' of 'scraped': 97486
Duplicated in column 'last_updated' of 'scraped': 16275

verified
Duplicated in column 'Extension Name' of 'verified': 1591
Duplicated in column 'Publisher' of 'verified': 12540
Duplicated in column 'Verified' of 'verified': 50998
Duplicated in column 'Install Count' of 'verified': 38880
Duplicated in column 'Source Code' of 'verified': 12916

vulnerable
Duplicated in column 'Extension Name' of 'vulnerable': 1333
Duplicated in column 'Repository Link' of 'vulnerable': 917
Duplicated in column 'Repository Name' of 'vulnerable

In [53]:
for key, val in raw_files.items():
    print(f"{key}")
    for col in val.columns:
        print (f"Total unique values in '{col}' of '{key}': {val[col].nunique()}")
    print()    

scraped
Total unique values in 'name' of 'scraped': 80232
Total unique values in 'id' of 'scraped': 82027
Total unique values in 'publisher' of 'scraped': 58758
Total unique values in 'version' of 'scraped': 3025
Total unique values in 'description' of 'scraped': 69055
Total unique values in 'categories' of 'scraped': 791
Total unique values in 'tags' of 'scraped': 31923
Total unique values in 'install_count' of 'scraped': 15239
Total unique values in 'rating' of 'scraped': 814
Total unique values in 'last_updated' of 'scraped': 82025

verified
Total unique values in 'Extension Name' of 'verified': 49409
Total unique values in 'Publisher' of 'verified': 38460
Total unique values in 'Verified' of 'verified': 2
Total unique values in 'Install Count' of 'verified': 12120
Total unique values in 'Source Code' of 'verified': 38083

vulnerable
Total unique values in 'Extension Name' of 'vulnerable': 31718
Total unique values in 'Repository Link' of 'vulnerable': 32134
Total unique values in '

### 2. Cleaning

Drop duplicated after scraping

In [54]:
# df_scraped[df_scraped.duplicated(subset=["name","publisher","version"])]
# df_verified[df_verified.duplicated(subset=["Extension Name","Publisher","Install Count"])]
# df_vulnerable[df_vulnerable.duplicated(subset=["Extension Name","Repository Link"])]
df_scraped = df_scraped.drop_duplicates(subset=["name","publisher","version"])
df_verified = df_verified.drop_duplicates(subset=["Extension Name","Publisher","Source Code"])
df_vulnerable = df_vulnerable.drop_duplicates(subset=["Extension Name","Repository Link"])

#### Clean column names and remove columns

In [56]:
raw_files = {"scraped": df_scraped, "verified": df_verified, "vulnerable": df_vulnerable}
for val in raw_files.values():
    val.columns = val.columns.str.strip().str.lower().str.replace(" ", "_")

In [59]:
df_scraped.columns, df_verified.columns, df_vulnerable.columns

(Index(['ext_name', 'ext_publisher', 'ext_version', 'ext_categories',
        'ext_tags', 'ext_install_count', 'ext_rating', 'ext_last_updated'],
       dtype='object'),
 Index(['ext_name', 'ext_publisher', 'verified', 'repository'], dtype='object'),
 Index(['ext_name', 'repository', 'total_vulnerabilities', 'critical', 'high',
        'medium', 'low'],
       dtype='object'))

In [58]:
df_scraped = df_scraped.drop(["id", "description"], axis = 1).add_prefix("ext_") #avoid overfitting for model training, add prefix to cols
df_verified = df_verified.drop(["install_count"],axis = 1).rename(columns = {"extension_name":"ext_name", "publisher": "repo_publisher", "source_code":"repository"}) # install_count of extension in this df not updated
df_vulnerable = df_vulnerable.drop(["repository_name", "critical_vulnerability_names", "high_vulnerability_names", "medium_vulnerability_names", "low_vulnerability_names"],axis = 1).rename(columns = {"extension_name": "ext_name", "repository_link":"repository"})



In [ ]:

# df_clean["repository"] = [x[:-4] if x.endswith(".git") else x for x in df_clean["repository"]]

#### Check null values and drop na

In [60]:
df_scraped = df_scraped.dropna(subset=["ext_publisher"])

In [63]:
df_verified#.isna().sum()

,ext_name,ext_publisher,verified,repository
0,Python,Microsoft,True,https://github.com/Microsoft/vscode-python.git
1,Jupyter,Microsoft,True,https://github.com/Microsoft/vscode-jupyter.git
2,Pylance,Microsoft,True,https://github.com/microsoft/pylance-release.git
3,C/C++,Microsoft,True,https://github.com/Microsoft/vscode-cpptools.git
4,Jupyter Keymap,Microsoft,True,https://github.com/Microsoft/vscode-jupyter-ke...
...,...,...,...,...
50995,pack-view,sydeny,False,NaN
50996,Riverpod Feature Generator,Hancod,False,https://github.com/farhanvk8/riverpod-feature-...
50997,showtimeofduydd,DuyDD230997,False,NaN
50998,Starlight i18n,HiDeoo,True,https://github.com/HiDeoo/starlight-i18n.git


In [ ]:
# df_verified[~((df_verified["repository"].str.contains("https://git|https://www.git|github.com",na=False,regex=True)) | (df_verified["repository"].isna()))]


In [ ]:
# df_verified_repo = df_verified[df_verified["repository"].isna()].reset_index(drop=True)
# df_verified_repo.to_csv("../data/raw/extensions_verified_repo.csv")
# df_verified_repo

In [43]:
df_vulnerable = df_vulnerable[~df_vulnerable["ext_name"].isna()]

#### Convert date


In [ ]:
# df_scraped["date"] = pd.to_datetime(df_scraped["ext_last_updated"], errors ="coerce",format="%Y-%m-%dT%H:%M:%SZ")
# df_scraped["year"] = df_scraped["date"].dt.year.astype("Int32")
# df_scraped["month"] = df_scraped["date"].dt.month.astype("Int32")
# df_scraped["day"] = df_scraped["date"].dt.day.astype("Int32")
# df_scraped

,ext_name,ext_publisher,ext_version,ext_categories,ext_tags,ext_install_count,ext_rating,ext_last_updated,date,year,month,day
0,Python,ms-python,2025.11.2025072901,Programming Languages;Debuggers;Data Science;M...,__ext_j2;__ext_jinja2;__web_extension;debugger...,177844528,4.192869,2025-07-29T10:50:24.31Z,NaT,<NA>,<NA>,<NA>
1,Pylance,ms-python,2025.7.100,Programming Languages,__web_extension;json;python,146473811,3.007722,2025-07-30T23:10:58.657Z,NaT,<NA>,<NA>,<NA>
2,Jupyter,ms-toolsai,2025.7.2025073101,Extension Packs;Data Science;Machine Learning;...,__ext_jl;__web_extension;debuggers;interactive...,93931981,2.700599,2025-07-31T09:54:06Z,2025-07-31 09:54:06,2025,7,31
3,C/C++,ms-vscode,1.26.3,Programming Languages;Snippets;Linters;Debugge...,C;C++;cpp;cuda-cpp;debuggers;IntelliSense;json...,85413454,3.426871,2025-06-25T19:02:16.243Z,NaT,<NA>,<NA>,<NA>
4,Python Debugger,ms-python,2025.11.2025072901,Debuggers,debugger;debuggers;debugpy;python,84033338,4.615385,2025-07-29T10:38:32.423Z,NaT,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
98292,𝐖𝐀𝐓𝐂𝐇 𝐓𝐡𝐞 𝐁𝐚𝐭𝐦𝐚𝐧 𝐌𝐨𝐯𝐢𝐞 𝐆𝐨𝐨𝐠𝐥𝐞𝐃𝐫𝐢𝐯𝐞-𝐇𝐃,TheB-a-t-m-a-nSubEnglish,0.0.1,Other,__web_extension,0,0.000000,2022-04-11T17:10:47.08Z,NaT,<NA>,<NA>,<NA>
98293,City Car Driving 125 Audio Dll High Quality,fuserpugzo,0.0.1,Other,NaN,0,0.000000,2022-03-15T19:02:37.72Z,NaT,<NA>,<NA>,<NA>
98295,Jurassic World: Il Dominio (2022) Streaming It...,add,0.9.9,Themes,__web_extension;color-theme;theme,0,0.000000,2022-04-07T15:09:15.58Z,NaT,<NA>,<NA>,<NA>
98296,PortableAutodeskAutoCAD2010 fillory,nahavox,0.0.1,Other,NaN,0,0.000000,2022-03-23T08:15:17.353Z,NaT,<NA>,<NA>,<NA>


In [71]:
df_scraped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81945 entries, 0 to 98297
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ext_name           81945 non-null  object        
 1   ext_publisher      81945 non-null  object        
 2   ext_version        81945 non-null  object        
 3   ext_categories     81945 non-null  object        
 4   ext_tags           57433 non-null  object        
 5   ext_install_count  81945 non-null  int64         
 6   ext_rating         81945 non-null  float64       
 7   ext_last_updated   81945 non-null  object        
 8   date               352 non-null    datetime64[ns]
 9   year               352 non-null    float64       
 10  month              352 non-null    float64       
 11  day                352 non-null    float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 8.1+ MB


#### Check duplicated and remove duplicates (deprecaded)

In [7]:
# df_scraped = df_scraped.drop_duplicates(subset=["name","publisher"])
# df_verified = df_verified.drop_duplicates(subset=["name","repository"])
# df_vulnerable = df_vulnerable.drop_duplicates(subset=["name","repository"])

In [ ]:
# raw_files = {"scraped": df_scraped, "verified": df_verified, "vulnerable": df_vulnerable}
# for key, val in raw_files.items():
#         print (f"Dimension of '{key}' after dropping null and duplicated: {val.shape}")

#### Combine raw df after cleaning 

In [76]:
df_scraped.head()

,ext_name,ext_publisher,ext_version,ext_categories,ext_tags,ext_install_count,ext_rating,ext_last_updated,date,year,month,day
0,Python,ms-python,2025.11.2025072901,Programming Languages;Debuggers;Data Science;M...,__ext_j2;__ext_jinja2;__web_extension;debugger...,177844528,4.192869,2025-07-29T10:50:24.31Z,NaT,<NA>,<NA>,<NA>
1,Pylance,ms-python,2025.7.100,Programming Languages,__web_extension;json;python,146473811,3.007722,2025-07-30T23:10:58.657Z,NaT,<NA>,<NA>,<NA>
2,Jupyter,ms-toolsai,2025.7.2025073101,Extension Packs;Data Science;Machine Learning;...,__ext_jl;__web_extension;debuggers;interactive...,93931981,2.700599,2025-07-31T09:54:06Z,2025-07-31 09:54:06,2025,7,31
3,C/C++,ms-vscode,1.26.3,Programming Languages;Snippets;Linters;Debugge...,C;C++;cpp;cuda-cpp;debuggers;IntelliSense;json...,85413454,3.426871,2025-06-25T19:02:16.243Z,NaT,<NA>,<NA>,<NA>
4,Python Debugger,ms-python,2025.11.2025072901,Debuggers,debugger;debuggers;debugpy;python,84033338,4.615385,2025-07-29T10:38:32.423Z,NaT,<NA>,<NA>,<NA>


In [77]:
df_verified.head()

,ext_name,ext_publisher,verified,repository
0,Python,Microsoft,True,https://github.com/Microsoft/vscode-python.git
1,Jupyter,Microsoft,True,https://github.com/Microsoft/vscode-jupyter.git
2,Pylance,Microsoft,True,https://github.com/microsoft/pylance-release.git
3,C/C++,Microsoft,True,https://github.com/Microsoft/vscode-cpptools.git
4,Jupyter Keymap,Microsoft,True,https://github.com/Microsoft/vscode-jupyter-ke...


In [78]:
df_vulnerable.head()

,ext_name,repository,total_vulnerabilities,critical,high,medium,low
0,Perl cpanfile,https://github.com/bayashi/perlcpanfile.git,0,0,0,0,0
1,Lingua Franca,https://github.com/lf-lang/vscode-lingua-franc...,1,0,0,0,1
2,AppEngine Theme,https://github.com/chazeprasad/appengine-theme,0,0,0,0,0
3,Firestore Rules,https://github.com/ChFlick/firecode,0,0,0,0,0
4,Comment Labels,https://github.com/jamespgilbert/comment-label...,0,0,0,0,0


In [74]:
df_pre_clean = pd.merge(df_verified,df_vulnerable, on=["ext_name", "repository"])
df_clean = pd.merge(df_scraped, df_pre_clean, on = "ext_name")
# df_clean["repository"] = [x[:-4] if x.endswith(".git") else x for x in df_clean["repository"]]
# df_clean.to_csv(config["data"]["clean"]["file_cleaned"])

In [75]:
df_clean.head()

,ext_name,ext_publisher_x,ext_version,ext_categories,ext_tags,ext_install_count,ext_rating,ext_last_updated,date,year,month,day,ext_publisher_y,verified,repository,total_vulnerabilities,critical,high,medium,low
0,Python,ms-python,2025.11.2025072901,Programming Languages;Debuggers;Data Science;M...,__ext_j2;__ext_jinja2;__web_extension;debugger...,177844528,4.192869,2025-07-29T10:50:24.31Z,NaT,<NA>,<NA>,<NA>,Microsoft,True,https://github.com/Microsoft/vscode-python.git,0,0,0,0,0
1,Pylance,ms-python,2025.7.100,Programming Languages,__web_extension;json;python,146473811,3.007722,2025-07-30T23:10:58.657Z,NaT,<NA>,<NA>,<NA>,Microsoft,True,https://github.com/microsoft/pylance-release.git,0,0,0,0,0
2,Jupyter,ms-toolsai,2025.7.2025073101,Extension Packs;Data Science;Machine Learning;...,__ext_jl;__web_extension;debuggers;interactive...,93931981,2.700599,2025-07-31T09:54:06Z,2025-07-31 09:54:06,2025,7,31,Microsoft,True,https://github.com/Microsoft/vscode-jupyter.git,0,0,0,0,0
3,C/C++,ms-vscode,1.26.3,Programming Languages;Snippets;Linters;Debugge...,C;C++;cpp;cuda-cpp;debuggers;IntelliSense;json...,85413454,3.426871,2025-06-25T19:02:16.243Z,NaT,<NA>,<NA>,<NA>,Microsoft,True,https://github.com/Microsoft/vscode-cpptools.git,11,0,0,0,11
4,Jupyter Keymap,ms-toolsai,1.1.2,Notebooks,__web_extension;keybindings;notebook-keymap,74084334,4.000000,2023-06-05T17:53:31.993Z,NaT,<NA>,<NA>,<NA>,Microsoft,True,https://github.com/Microsoft/vscode-jupyter-ke...,17,0,0,9,8


In [ ]:
df = pd.read_csv(config["data"]["clean"]["file_cleaned"])
df.shape